# OpenAI Function Calling: Weather Tool Demo

This workbook shows how to use OpenAI’s function calling (tools) to let a model decide when to call a local Python function to fetch data, then incorporate the result into its final answer.

- Client: OpenAI (`OPENAI_API_KEY` required)
- Model: `gpt-4o-mini`
- Tool: `get_current_weather(location, unit)` returning a small weather dict
- Flow: user question → model proposes tool call → Python executes → model uses the result to answer

Docs: https://platform.openai.com/docs/guides/function-calling

In [ ]:
from openai import OpenAI

client = OpenAI()

In [9]:
# Baseline: Ask the model without tools
prompt = "What's the weather like in Philly, PA?"
baseline = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
)
print(baseline.choices[0].message.content)

I can't provide real-time weather updates. For the current weather in Philadelphia, PA, I recommend checking a weather website or app for the latest information.


In [10]:
import json

# Toy local function
def get_current_weather(location: str, unit: str = "fahrenheit"):
    """Gets current weather conditions for a location (mock)."""
    return {
        "location": location,
        "temperature": 61.0,
        "unit": unit,
        "feels_like": 59.0,
        "condition": "Partly cloudy",
        "humidity": 30,
        "wind_speed": "2.2 mph",
    }

# Tool definition (JSON Schema)
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Gets the current weather conditions for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and state, e.g., Philadelphia, PA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit",
                },
            },
            "required": ["location"],
        },
    },
}

# Automatic tool calling loop
messages = [{"role": "user", "content": prompt}]
first = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=[weather_tool],
    tool_choice="auto",
)
msg = first.choices[0].message

# If the model requests tool(s), execute and send results back
if getattr(msg, "tool_calls", None):
    messages.append({"role": "assistant", "content": msg.content or "", "tool_calls": msg.tool_calls})
    for call in msg.tool_calls:
        if call.function.name == "get_current_weather":
            args = json.loads(call.function.arguments or "{}")
            result = get_current_weather(**args)
            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "content": json.dumps(result),
            })
    # Ask the model to use the tool outputs
    final = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    print(final.choices[0].message.content)
else:
    # No tools requested; just print the model's answer
    print(msg.content)

The weather in Philadelphia, PA is currently 61°F and partly cloudy. It feels like 59°F, with a humidity level of 30% and a light wind blowing at 2.2 mph.


In [ ]:
from pprint import pprint
import src.current_weather as cw

city = "Philadelphia, PA"
weather_info = cw.get_current_weather(city)
pprint(weather_info, width=100, compact=True)

('{"location": "Philadelphia, PA", "temperature": 39.0, "unit": "fahrenheit", "feels_like": 33.5, '
 '"condition": "Light rain", "humidity": 82, "wind_speed": "7.8 mph"}')


In [19]:
# Wrap the module's weather function for tool calling

def api_get_current_weather(location: str, unit: str = "fahrenheit"):
    return cw.get_current_weather(location)

# Quick local test
pprint(api_get_current_weather(city))

('{"location": "Philadelphia, PA", "temperature": 39.0, "unit": "fahrenheit", '
 '"feels_like": 33.5, "condition": "Light rain", "humidity": 82, "wind_speed": '
 '"7.8 mph"}')


In [23]:
import json

# Define tool spec referencing the wrapper above
weather_tool = {
    "type": "function",
    "function": {
        "name": "api_get_current_weather",
        "description": "Gets the current weather conditions for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    },
}

prompt = f"What's the weather like in {city}?"
messages = [{"role": "user", "content": prompt}]

first = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=[weather_tool],
    tool_choice="auto",
)
msg = first.choices[0].message

if getattr(msg, "tool_calls", None):
    messages.append({"role": "assistant", "content": msg.content or "", "tool_calls": msg.tool_calls})
    for call in msg.tool_calls:
        if call.function.name == "api_get_current_weather":
            args = json.loads(call.function.arguments or "{}")
            result = api_get_current_weather(**args)
            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "content": json.dumps(result),
            })
    final = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    pprint(final.choices[0].message.content)
else:
    pprint(msg.content)

('The current weather in Philadelphia, PA is about 39°F with light rain. It '
 'feels like 33.5°F due to the humidity, which is at 82%. There is a wind '
 'speed of 7.8 mph.')


### What just happened in the tool-calling flow
- Model saw the question and auto-requested the tool `api_get_current_weather`.
- Python executed the tool and returned structured weather data.
- The assistant then used that tool output to craft the final answer.
- Key pieces: `tools=[weather_tool]`, check `tool_calls`, execute locally, send results back, ask model again.